In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import tensorflow as tf
import edward as ed
from edward.models import Normal

path = "./Input/"  #Insert path here
database = path + 'database.sqlite'

In [2]:
import numpy as np
import pandas as pd
import sqlite3

path = "./Input/"  #Insert path here
database = path + 'database.sqlite'

conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

detailed_matches = pd.read_sql("""SELECT Match.id, 
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season, 
                                        stage, 
                                        date,
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        home_team_goal, 
                                        away_team_goal                                        
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                where country_name = 'England' and season = '2008/2009'
                                ORDER by date
                                ;""", conn)
detailed_matches

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal
0,1730,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Arsenal,West Bromwich Albion,1,0
1,1731,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Sunderland,Liverpool,0,1
2,1732,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,West Ham United,Wigan Athletic,2,1
3,1734,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Everton,Blackburn Rovers,2,3
4,1735,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1
5,1736,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Bolton Wanderers,Stoke City,3,1
6,1737,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Hull City,Fulham,2,1
7,1729,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,Manchester United,Newcastle United,1,1
8,1733,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,Aston Villa,Manchester City,4,2
9,1738,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,Chelsea,Portsmouth,4,0


In [3]:
detailed_matches = pd.read_sql("""SELECT Match.id, 
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season, 
                                        stage, 
                                        date,
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        home_team_goal, 
                                        away_team_goal                                        
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                where country_name = 'England' and season = '2008/2009'
                                ORDER by date
                                ;""", conn)
detailed_matches

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal
0,1730,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Arsenal,West Bromwich Albion,1,0
1,1731,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Sunderland,Liverpool,0,1
2,1732,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,West Ham United,Wigan Athletic,2,1
3,1734,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Everton,Blackburn Rovers,2,3
4,1735,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1
5,1736,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Bolton Wanderers,Stoke City,3,1
6,1737,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Hull City,Fulham,2,1
7,1729,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,Manchester United,Newcastle United,1,1
8,1733,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,Aston Villa,Manchester City,4,2
9,1738,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,Chelsea,Portsmouth,4,0


In [4]:
# Make a matrix of nxn, matchups of all the teams
teams = detailed_matches.home_team.unique()
teams_dict = dict(zip(teams, range(len(teams))))

n = len(teams)
matchup = np.zeros((n,n))

for index, row in detailed_matches.iterrows():
    home = teams_dict[row.home_team]
    away = teams_dict[row.away_team]
    score = row.home_team_goal - row.away_team_goal
    matchup[home, away] += score
    matchup[away, home] -= score

In [5]:
#latent dimension representation
K = 2
z = Normal(loc=tf.zeros([n, K]), scale=tf.ones([n, K]))
xp = tf.tile(tf.reduce_sum(z, 1, keepdims=True), [1, n])
xp = xp - tf.transpose(xp)

x = Normal(xp, abs(xp))

In [6]:
x

<ed.RandomVariable 'Normal_1/' shape=(20, 20) dtype=float32>

In [7]:
qz = Normal(loc=tf.get_variable("qz/loc", [n, K]),
            scale=tf.nn.softplus(tf.get_variable("qz/scale", [n, K])))
inference = ed.KLqp({z: qz}, data={xp: matchup})

C:\Users\aanic\Anaconda3\envs\pp_project\lib\site-packages\edward\util\random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [8]:
inference.run(n_iter=100)

100/100 [100%] ██████████████████████████████ Elapsed: 0s | Loss: 0.000


In [16]:
x.eval()[0,]

array([ 0.        ,  0.8336288 , -4.9802237 ,  0.24382722,  4.7438765 ,
        0.10553765,  5.261629  ,  2.5477595 ,  0.21423864,  2.8583825 ,
       -1.3512446 ,  0.33692968,  1.03979   ,  3.5611627 ,  4.1573462 ,
        1.2821646 ,  0.21137987,  0.09592962,  2.2002382 ,  1.8892424 ],
      dtype=float32)

In [12]:
matchup[1,]

array([ 0.,  0., -3., -5.,  2., -3.,  4., -2., -2., -6., -3.,  1.,  1.,
        1.,  1.,  1.,  1., -4., -1., -3.])

In [17]:
with tf.Session() as sess:
    print(tf.reduce_sum(tf.subtract(xp,matchup)).eval())

0.0


In [13]:
with tf.Session() as sess:
    print(xp.eval())
    print(abs(xp).eval())

[[ 0.          0.68981576  1.333881   ...  1.9661767   0.735762
   2.3585865 ]
 [-0.68981576  0.          0.64406526 ...  1.276361    0.04594624
   1.6687708 ]
 [-1.333881   -0.64406526  0.         ...  0.6322957  -0.598119
   1.0247055 ]
 ...
 [-1.9661767  -1.276361   -0.6322957  ...  0.         -1.2304147
   0.3924098 ]
 [-0.735762   -0.04594624  0.598119   ...  1.2304147   0.
   1.6228245 ]
 [-2.3585865  -1.6687708  -1.0247055  ... -0.3924098  -1.6228245
   0.        ]]
[[0.         0.65217185 1.8004344  ... 1.7904303  2.007286   1.6187985 ]
 [0.65217185 0.         2.4526062  ... 1.1382585  1.3551142  0.9666267 ]
 [1.8004344  2.4526062  0.         ... 3.5908647  3.8077202  3.4192328 ]
 ...
 [1.7904303  1.1382585  3.5908647  ... 0.         0.21685576 0.1716317 ]
 [2.007286   1.3551142  3.8077202  ... 0.21685576 0.         0.38848746]
 [1.6187985  0.9666267  3.4192328  ... 0.1716317  0.38848746 0.        ]]


In [11]:
xp

<tf.Tensor 'sub:0' shape=(34, 34) dtype=float32>